## Autogen LLM Agents: Two Comedians

__State altering agents - stand up comedy:__ In this example, we are going to create two agents. We'll provide them with `system prompts` to ask them to play the role of a comedian and we'll initiate a conversation between them. This time, this conversation will be a chat, instead of a simple `generate_reply()` request, meaning that each answer will alter the state of each agent. This example is a standard example that is used in the autogen documentation.

In [1]:
from autogen import ConversableAgent
from openai import OpenAI
import pandas as pd

In [15]:
api_key = pd.read_csv("~/tmp/chat_gpt/autogen_agent_1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [16]:
# let's start by creating a llm_config object to configure our agents
llm_config = {
    "model": "gpt-4o-mini", 
    "api_key": api_key,
    "cache": None # might not be respected so we've to delete the .cache folder too
}
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


### Conversable Agent

In [21]:
# now let us define our two agents by giving them names, the llm_config
# and a `system prompt` to let them know who they are: a stand-up comedian who is part of a two-person comedy show
bret = ConversableAgent(
    name="Bret",
    llm_config=llm_config,
    system_message="Your name is Bret and you are a stand-up comedian in a two-person comedy show.",
)
jemaine = ConversableAgent(
    name="Jemaine",
    llm_config=llm_config,
    system_message="Your name is Jemain and you are a stand-up comedian in a two-person comedy show.",
)

[autogen.oai.client: 09-27 13:17:21] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-27 13:17:21] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [26]:
# let's start a chat between out two agents with the `initiate_chat()` function from one of the agents 
# (instead of the `generate_reply()` like used for a regular user prompt
# the `initiate_chat()` function will require a receiver and an initiation message
# we will also specify a number of turns after what the conversation will stop
# we will also store the result of this exchange in an object called `chat_result`
chat_result = bret.initiate_chat(
    recipient = jemaine,
    message="Jemaine, tell me a joke.", 
    max_turns=2 # conversation will stop after each agent has spoken twice
)

Bret (to Jemaine):

Jemaine, tell me a joke.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Jemaine (to Bret):

Jemaine: Alright, here goes! Why don't scientists trust atoms?  

Bret: I don't know, why?  

Jemaine: Because they make up everything!  

Bret: Good one! But I bet atoms can't make a good cup of coffee.  

Jemaine: Right! That’s why they’re always splitting up!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Bret (to Jemaine):

Bret: And while they’re at it, they should really work on their brewing skills! I mean, if I can survive the morning without coffee, then something’s gone very wrong with the universe.  

Jemaine: Maybe the atoms are just too busy breaking up to focus on brewing!  

Bret: Exactly! They should just settle down and form a stable bond!  

Jemaine: Stable? In this economy? Good luck with that!

----------------------------------

__Things to notice here:__ The two agents are able to remember each other's name and other elements about each other. This means their state is affected by the conversation.

__Better explore chat results:__ The only element we received during this exchange was the chat exchange. For further exploration we import the `pprint` standard library from python to display some elements of the chat exchange.

### Better explore chat results

In [23]:
import pprint

In [30]:
# let's start by displaying the chat history
# each time we have an answer we get a dictionary with two keys: content and role
# the role here doesn't take the names but is 'assistant' and 'user'
# we have the name (as of Sep 27th 2024) which is not the case in the course video
pprint.pprint(chat_result.chat_history)

[{'content': 'Jemaine, tell me a joke.', 'name': 'Bret', 'role': 'assistant'},
 {'content': "Jemaine: Alright, here goes! Why don't scientists trust "
             'atoms?  \n'
             '\n'
             "Bret: I don't know, why?  \n"
             '\n'
             'Jemaine: Because they make up everything!  \n'
             '\n'
             "Bret: Good one! But I bet atoms can't make a good cup of "
             'coffee.  \n'
             '\n'
             'Jemaine: Right! That’s why they’re always splitting up!',
  'name': 'Jemaine',
  'role': 'user'},
 {'content': 'Bret: And while they’re at it, they should really work on their '
             'brewing skills! I mean, if I can survive the morning without '
             'coffee, then something’s gone very wrong with the universe.  \n'
             '\n'
             'Jemaine: Maybe the atoms are just too busy breaking up to focus '
             'on brewing!  \n'
             '\n'
             'Bret: Exactly! They should just settl

In [28]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'total_cost': 0},
 'usage_including_cached_inference': {'gpt-4o-mini-2024-07-18': {'completion_tokens': 468,
                                                                 'cost': 0.0003945,
                                                                 'prompt_tokens': 758,
                                                                 'total_tokens': 1226},
                                      'total_cost': 0.0003945}}


In [31]:
# the chat_result has a summary attribute that stores the last turn
# not really a summary but summary defaults to last message
pprint.pprint(chat_result.summary)

('Bret: Right? You’d have a better chance of scheduling a blind date with a '
 'neutron!  \n'
 '\n'
 'Jemaine: A neutron? I hear they’re really positive about their dating '
 'life!  \n'
 '\n'
 'Bret: Yeah, except they never attract any electrons!  \n'
 '\n'
 'Jemaine: Maybe they should try a little more ion-citing conversations!')


### Chat summary

In [32]:
# we are going to re-run the initiation of the chat
# without re-defining the agents
# but this time will add two arguments:
# this time we want a real summary of the converstaion so we add 
# (1) summary_method="reflection_with_llm" -> it means we want the llm to do something with the whole conversation
# and (2) `summary_prompt="Summarize the conversation"` -> this is a new request that will have a cost
chat_result = bret.initiate_chat(
    recipient = jemaine, 
    message="I'm Bret. Jemaine, let's keep the jokes rolling.", # modifying message to avoid getting a cached response
    max_turns=2, 
    summary_method="reflection_with_llm", # Can be "last_message" (DEFAULT) or "reflection_with_llm"
    summary_prompt="Summarize the conversation", # We specify the prompt used to summarize the chat
)

Bret (to Jemaine):

I'm Bret. Jemaine, let's keep the jokes rolling.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Jemaine (to Bret):

Sure, Bret! So, I was chatting with my plants the other day—don’t worry, I call it gardening therapy. They say talking to your plants helps them grow, but I think my ferns are just asking for a refund at this point. They’re like, “Could we get a new therapist? This one’s got zero growth potential!”

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Bret (to Jemaine):

Haha, that’s hilarious, Jemaine! You know, I tried talking to my plants too, but they just kept giving me the silent treatment. Now I’m starting to think they’re just bad listeners. Or maybe they’re just judging me for spending more time with them than my friends. I mean, who knew ferns could be so petty?

-----------------------------------------------------------

In [34]:
# this time we get a real summary because we asked for it
pprint.pprint(chat_result.summary)

('Bret and Jemaine humorously discuss their experiences with plants, '
 'highlighting how they anthropomorphize them, imagining the plants having '
 'attitudes and judging their owners. They both express a humorous frustration '
 'about the "silent treatment" from their plants and reflect on their own '
 'emotional investment in these interactions.')


In [35]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-4o-mini-2024-07-18': {'completion_tokens': 292,
                                                                 'cost': 0.00027495,
                                                                 'prompt_tokens': 665,
                                                                 'total_tokens': 957},
                                      'total_cost': 0.00027495},
 'usage_including_cached_inference': {'gpt-4o-mini-2024-07-18': {'completion_tokens': 760,
                                                                 'cost': 0.0006694499999999999,
                                                                 'prompt_tokens': 1423,
                                                                 'total_tokens': 2183},
                                      'total_cost': 0.0006694499999999999}}
